In [ ]:
!sudo apt-get install -y build-essential
!pip install tree-sitter
!git clone https://github.com/tree-sitter/tree-sitter-python
!cd tree-sitter-python && gcc -o parser.so -I./src src/parser.c src/scanner.c -shared -Os -lstdc++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp310-cp310-linux_x86_64.whl size=424708 sha256=03d8906e43db8182256739c722bdd776adfb516cf393fbdbf4284d4f57280788
  Stored in directory: /root/.cache/pip/wheels/e6/d0/7a/a108b30f6615a71ca3a07ced1b149509d437a60c9d64820723
Successfully built tree-sitter
Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 2934, done.
remote: Counting objects: 100% (1155/1155), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 2934 (delta 1022), reused 1096 (delta 996), pack-reused 1

In [ ]:
!gcc -o /content/tree-sitter-python/parser.so -I /content/tree-sitter-python/src /content/tree-sitter-python/src/parser.c /content/tree-sitter-python/src/scanner.c -shared -Os -lstdc++


In [ ]:
import tree_sitter

# Load the language's grammar
parser_path = '/content/tree-sitter-python/parser.so'

PYTHON_LANGUAGE = tree_sitter.Language(parser_path, 'python')

# Create a parser
parser = tree_sitter.Parser()
parser.set_language(PYTHON_LANGUAGE)



In [ ]:
import requests

# URL of the raw file on GitHub
# url = ''

# Download the file
# response = requests.get(url)

# Check if the request was successful

# if response.status_code == 200:
#     code = response.text  # Here is where 'code' is defined
# else:
#     print('Failed to download file')
#     code = ''

# print(code)

file = open("graphine.py","r")
content = file.read()
tree = parser.parse(bytes(content, "utf8"))
print(tree.root_node)

<Node type=module, start_point=(0, 0), end_point=(98, 8)>


* 'module': Represents the entire source file.
* 'function_definition': Represents a function definition.
* 'parameters': Represents the parameters of a function.
* 'call_expression': Represents a function call.
* 'argument_list': Represents the arguments in a function call.
* 'identifier': Represents an identifier, such as a variable or function name.
* 'string': Represents a string literal.
* 'integer': Represents an integer literal.
* 'if_statement': Represents an if statement.
* 'for_statement': Represents a for loop.
* 'while_statement': Represents a while loop.
* 'return_statement': Represents a return statement.
* 'expression_list': Represents a list of expressions.
* 'attribute': Represents an attribute access, like obj.attr.
* 'binary_operator': Represents a binary operation, like addition or multiplication.
* 'unary_operator': Represents a unary operation, like negation.




In [ ]:
import networkx as nx

# Create a graph to store caller-callee relationships
graph = nx.DiGraph()
def get_function_name(node):
    # print(node.children)
    # Assuming the function name is the first child of the function definition
    function_name_node = node.children[1]
    # print("num", len(node.children))
    # for i in range(len(node.children)):
    #   print(node.children[i].type)



    if function_name_node.type == 'identifier':
        print(function_name_node.text)
        return function_name_node.text
    return None

def get_callee_name(node):
    # Assuming the callee is the first child of the call expression
    callee_node = node.children[0]
    if callee_node.type == 'identifier':
        return callee_node.text
    elif callee_node.type == 'attribute':
        # Handle cases like obj.method()
        return get_attribute_name(callee_node)
    return None

def get_attribute_name(node):
    # Recursive function to handle attributes like obj.attr.method()
    if node.type == 'attribute':
        base_name = get_attribute_name(node.children[0])
        attr_name = node.children[2].text
        return f"{base_name}.{attr_name}"
    elif node.type == 'identifier':
        return node.text
    return None

# Function to traverse the AST and identify function definitions and calls
def traverse_tree(node, parent_function=None):
    # print(node)
    if node.type == 'function_definition':
        function_name = get_function_name(node)
        graph.add_node(function_name, type='function')

        # If inside another function, add a caller-callee edge
        if parent_function:
            graph.add_edge(parent_function, function_name, relation='caller-callee')

        # Traverse children within the context of this function
        for child in node.children:
            traverse_tree(child, parent_function=function_name)
    elif node.type == 'call_expression':
        callee_name = get_callee_name(node)
        if parent_function and callee_name in graph.nodes:
            graph.add_edge(parent_function, callee_name, relation='caller-callee')

        # Traverse children without changing the parent function context
        for child in node.children:
            traverse_tree(child, parent_function=parent_function)
    else:
        # Traverse children without changing the parent function context
        for child in node.children:
            traverse_tree(child, parent_function=parent_function)

# Start the traversal
traverse_tree(tree.root_node)

b'graphs'
b'functions'


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "WizardLM/WizardCoder-15B-V1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# checkpoint = "Salesforce/instructcodet5p-16b"
# device = "cuda" # for GPU usage or "cpu" for CPU usage

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
#                                               torch_dtype=torch.float16,
#                                               low_cpu_mem_usage=True,
#                                               trust_remote_code=True).to(device)

# encoding = tokenizer("def print_hello_world():", return_tensors="pt").to(device)
# encoding['decoder_input_ids'] = encoding['input_ids'].clone()
# outputs = model.generate(**encoding, max_length=15)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))




In [ ]:
def transform_log_function(code):
    # Define the transformation task
    task = f"Add a 'loglevel' argument to the log function in the following code: {code}"

    # Encode and run the model
    inputs = tokenizer.encode(task, return_tensors="pt")
    outputs = model.generate(inputs)
    transformed_code = tokenizer.decode(outputs[0])

    return transformed_code

In [ ]:
def correct_log_calls(code):
    # Find all calls to the log function
    log_calls = find_log_calls(graph)

    # Transform each call
    for call in log_calls:
        code = transform_log_call(code, call)

    return code


In [ ]:
def highlight_edits_with_model(ast_tree, previous_edits=None):
    # Load the pre-trained model and tokenizer
    model_name = "Salesforce/instructcodet5p-16b"
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Convert the AST tree to text representation (you'll need to implement this conversion)
    ast_text = ast_to_text(ast_tree)

    # Apply previous edits if available
    if previous_edits:
        ast_text = apply_previous_edits(ast_text, previous_edits)

    # Encode the AST text
    input_ids = tokenizer.encode(ast_text, return_tensors="pt")

    # Generate the highlighted output
    output = model.generate(input_ids)

    # Decode and return the highlighted text
    highlighted_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return highlighted_text

def ast_to_text(ast_tree):
    # Convert the abstract syntax tree to text representation
    # You need to implement this function based on your use case
    pass

def apply_previous_edits(text, previous_edits):
    # Apply previous edits to the text
    # You need to implement this function based on your use case
    pass